# eof - EO Fetch Demo

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/profLewis/eof/blob/main/notebooks/demo.ipynb)

This notebook demonstrates all `eof` functionality:

1. Install and check available sources
2. Fetch Sentinel-2 data (defaults to AWS, no credentials needed)
3. Inspect the returned S2Result
4. Plot NDVI time series and maps
5. Test the credential and speed-probe utilities

## 1. Install eof

In [ ]:
!pip install -q https://github.com/profLewis/eof/archive/refs/heads/main.zip

## 2. Optional: set up credentials

Skip these cells to use **AWS Earth Search** (no credentials needed).

Two options for configuring other data sources:

**Option A**: Upload a previously saved `config.json` (run the next cell)

**Option B**: Enter credentials manually (run the cell after that)

- **CDSE**: [Get S3 keys](https://eodata.dataspace.copernicus.eu) or use your [login](https://dataspace.copernicus.eu)
- **GEE**: Run `earthengine authenticate` in a terminal, or follow the prompt below

In [ ]:
# Option A: Upload a saved config.json
import eof
from pathlib import Path
import shutil

try:
    from google.colab import files
    print("Upload your config.json (or click Cancel to skip):")
    uploaded = files.upload()
    if "config.json" in uploaded:
        config_dir = Path(eof.CONFIG_FILE).parent
        config_dir.mkdir(parents=True, exist_ok=True)
        Path(eof.CONFIG_FILE).write_bytes(uploaded["config.json"])
        config = eof.load_config()
        print(f"\nConfig installed at {eof.CONFIG_FILE}")
        eof.print_config_status()
    else:
        print("No config.json uploaded — skipped.")
except ImportError:
    # Not running in Colab — check if config already exists
    if Path(eof.CONFIG_FILE).exists():
        print(f"Config already exists: {eof.CONFIG_FILE}")
        eof.print_config_status()
    else:
        print("Not in Colab. Place config.json at ~/.eof/config.json manually,")\
        ; print("or enter credentials in the next cell.")

In [ ]:
# Option B: Enter credentials manually (press Enter to skip any prompt)
import os
import getpass

print("Manual credential setup (press Enter to skip any)")
print("=" * 50)

config = eof.load_config()
changed = False

# CDSE S3 credentials
print("\n--- CDSE (S3 keys) ---")
s3_key = getpass.getpass("CDSE S3 Access Key (Enter to skip): ")
if s3_key:
    s3_secret = getpass.getpass("CDSE S3 Secret Key: ")
    os.environ["CDSE_S3_ACCESS_KEY"] = s3_key
    os.environ["CDSE_S3_SECRET_KEY"] = s3_secret
    config["cdse"]["s3_access_key"] = s3_key
    config["cdse"]["s3_secret_key"] = s3_secret
    changed = True
    print("CDSE S3 credentials set.")
else:
    print("Skipped CDSE.")

# GEE
print("\n--- Google Earth Engine ---")
try:
    import ee
    ee.Initialize()
    print("GEE: already authenticated.")
except Exception:
    gee_auth = input("Authenticate with GEE? [y/N]: ").strip().lower()
    if gee_auth == "y":
        import ee
        ee.Authenticate()
        ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com")
        print("GEE: authenticated.")
    else:
        print("Skipped GEE.")

# Save config file
if changed:
    eof.save_config(config)
    print(f"\nCredentials saved to {eof.CONFIG_FILE}")
else:
    print("\nNo new credentials entered.")

In [ ]:
# Download config.json to reuse on other machines / sessions
config_path = Path(eof.CONFIG_FILE)
if config_path.exists():
    try:
        from google.colab import files
        files.download(str(config_path))
        print(f"Downloading {config_path}")
    except ImportError:
        print(f"Config file: {config_path}")
        print("Copy it to ~/.eof/config.json on other machines to reuse credentials.")
else:
    print("No config file yet — enter or upload credentials first.")

## 3. Check available data sources

In [ ]:
eof.print_config_status()
print(f'\nAvailable sources: {eof.get_available_sources()}')
print(f'Test GeoJSON: {eof.TEST_GEOJSON}')

## 4. Fetch Sentinel-2 data

With no credentials configured, `source='auto'` defaults to **AWS Earth Search** (free, no auth).
If you entered credentials above, `auto` will pick the best available source.

The bundled test field (`eof.TEST_GEOJSON`) is a South African wheat field.

In [ ]:
result = eof.get_s2_data(
    start_date='2022-07-15',
    end_date='2022-11-30',
    geojson_path=eof.TEST_GEOJSON,
)

print(f'\nResult type: {type(result).__name__}')
print(f'reflectance shape: {result.reflectance.shape}  (images, bands, H, W)')
print(f'uncertainty shape:  {result.uncertainty.shape}')
print(f'angles shape:       {result.angles.shape}  [SZA, VZA, RAA]')
print(f'doys:               {result.doys}')
print(f'mask shape:         {result.mask.shape}')
print(f'geotransform:       {result.geotransform}')
print(f'N images:           {result.reflectance.shape[0]}')
print(f'Valid pixels:       {(~result.mask).sum()} / {result.mask.size}')

## 5. NDVI time series

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# NDVI = (NIR - Red) / (NIR + Red)  — bands 6 (B08) and 2 (B04)
nir = result.reflectance[:, 6]
red = result.reflectance[:, 2]
ndvi = (nir - red) / (nir + red)

mean_ndvi = np.nanmean(ndvi, axis=(1, 2))

plt.figure(figsize=(12, 4))
valid = np.isfinite(mean_ndvi)
plt.plot(result.doys[valid], mean_ndvi[valid], '--o', label='Mean NDVI')
plt.xlabel('Day of Year')
plt.ylabel('NDVI')
plt.title('Sentinel-2 NDVI Time Series (AWS)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. NDVI maps

In [ ]:
n_images = len(result.doys)
ncols = min(5, n_images)
nrows = int(np.ceil(n_images / ncols))

fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4 * nrows))
if nrows == 1:
    axs = np.atleast_2d(axs)
axs_flat = axs.ravel()

for i in range(n_images):
    ndvi_map = np.full(result.mask.shape, np.nan)
    ndvi_map[~result.mask] = ndvi[i][~result.mask]
    im = axs_flat[i].imshow(ndvi_map, vmin=0, vmax=1, cmap='RdYlGn')
    axs_flat[i].set_title(f'DOY {result.doys[i]}')
    axs_flat[i].axis('off')

for i in range(n_images, len(axs_flat)):
    axs_flat[i].axis('off')

fig.colorbar(im, ax=axs, shrink=0.6, label='NDVI')
plt.suptitle('NDVI Maps', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Backward-compatible interface

The `get_s2_official_data()` function returns a 7-tuple matching ARC's legacy interface.

In [ ]:
s2_refs, s2_uncs, s2_angles, doys, mask, geotransform, crs = eof.get_s2_official_data(
    start_date='2022-07-15',
    end_date='2022-11-30',
    geojson_path=eof.TEST_GEOJSON,
    source='aws',
)

print(f's2_refs shape:   {s2_refs.shape}')
print(f's2_angles shape: {s2_angles.shape}')
print(f'doys:            {doys}')
print(f'mask shape:      {mask.shape}')

## 8. Speed probe

Time a single-band download from each available source.

In [ ]:
speeds = eof.probe_download_speed(eof.TEST_GEOJSON)

## 9. Validation

In [ ]:
print('Validation checks:')
print(f'  reflectance dtype:  {result.reflectance.dtype} (should be float32)')
print(f'  reflectance range:  [{np.nanmin(result.reflectance):.4f}, {np.nanmax(result.reflectance):.4f}]')
print(f'  10 bands:           {result.reflectance.shape[1] == 10}')
print(f'  N dates == len(doys): {result.reflectance.shape[0] == len(result.doys)}')
print(f'  mask is boolean:    {result.mask.dtype == bool}')
print(f'  angles shape (3,N): {result.angles.shape[0] == 3}')

assert result.reflectance.dtype == np.float32
assert result.reflectance.shape[1] == 10
assert result.angles.shape[0] == 3
assert np.nanmax(result.reflectance) < 1.5, 'Reflectance values too high'
assert np.nanmin(result.reflectance) >= 0, 'Negative reflectance'
print('\nAll checks passed!')